In [6]:
from src.database.vector_database import VectorRetriever
import asyncio
import json



In [11]:
model_name = "sentence-transformers/all-MiniLM-L6-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}



vector_retriever = VectorRetriever(model_name = model_name, model_kwargs= model_kwargs, encode_kwargs=encode_kwargs, overwrite=False)
vector_retriever.initalize_pinecone_store(index_name="chef-app")# dont pass documents if you want to initialize an existing vector store

embedder initialized with dimension:  384
pinecone vector store initialized successfully, and ready for retrieval!


In [21]:
import asyncio
import json

# Asynchronously perform similarity searches and process results
async def perform_search(vector_retriever, query, k, where_document):
    results = await asyncio.to_thread(
        vector_retriever.similarity_search, query=query, k=k, where_document=where_document
    )
    for result in results:
        print(json.dumps(result, indent=4))
        print("\n\n")


# Define the main coroutine which will run all searches concurrently
async def main():

    
    ingredients_list = ["cup butter", "plain yogurt", "sugar", "1 egg", "vanilla"]
    where_document_condition = {
        "$and": [
            {"$contains": "dinner"},
            {"$contains": "Healthy"}
        ]
    }
    
    # Create a list of search tasks
    tasks = [
        asyncio.create_task(perform_search(vector_retriever, ingredients_list, 3, where_document_condition))
        for _ in range(500)
    ]

    
    # Run all tasks concurrently
    await asyncio.gather(*tasks)

# Run the main coroutine
await main()


{
    "recipe_name": "Buttercream Frosting",
    "recipe_ingredients": "['\u00bd cup butter, softened', '\u215b teaspoon salt', \"4 cups confectioners' sugar\", '2  egg yolks, beaten', '1 teaspoon vanilla extract', '2 tablespoons milk']",
    "recipe_directions": "['Cream butter until soft. Add salt and part of the sugar gradually, blending after each addition. Then add egg yolks and vanilla; blend well. Add remaining sugar, alternately with milk, until preferred consistency for spreading is reached, beating after each addition until smooth.']",
    "recipe_nutrition_details": "{'Calories': '352', 'Total Fat': 'None', 'Carbohydrates': 'None', 'Protein': '1g'}",
    "recipe_tags": "['Recipes', 'Desserts', 'Frostings and Icings', 'Buttercream']",
    "recipe_image_url": "https://www.allrecipes.com/thmb/p9kfMAZxoCxbU5wOx2VmfQN3-7k=/282x188/filters:no_upscale():max_bytes(150000):strip_icc():focal(124x0:126x2):format(webp)/216737-00e976769574434d8b661de2a998d734.jpg",
    "recipe_url": "htt

# it took 1.5 seconds for 10 calls/sec